<a href="https://colab.research.google.com/github/tyslas/CS5265-tyslas-nfl-spread-line-outcomes/blob/main/NFLSpreadAndLineOutcomes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Project to predict future NFL Spread & Line outcomes

## Author: Tito Yslas

### Background
I enjoy watching the NFL and playing fantasy football. I also like to place bets on games using apps like FanDuel. The purpose of this project is to increase my understanding of the NFL betting market and possibly create a machine learning model to give myself an edge next season.

### Project Description
I found a dataset from Kaggle titled [NFL scores and betting data](https://www.kaggle.com/datasets/tobycrabtree/nfl-scores-and-betting-data?resource=download). This dataset has over 13,000 samples and 17 features. The goal is to use this dataset to train a model that will predict the winner of a given game.

### Performance Metric
For my performance metric, I will aim for the model to have at least 70% accuracy.

## Import Libraries

In [1]:
import pandas as pd
import os
import seaborn as sns
import matplotlib.pyplot as plt

## Load Data

### Data Dictionary
- schedule_date: Date that the game took place. This column is a date in the format MM/DD/YYYY
- schedule_season: Year of the season began. NFL seasons start in the fall and end before the spring of the following year. So the 2023 season will refer to the years 2023-2024. This column is a number in the format YYYY
- schedule_week: Week of the NFL season. This column is either a number during regular season weeks or a string in playoff weeks. For the purposes of this project the playoff weeks will be converted to numbers and the schedule_playoff column will be used to determine whether the week is a playoff game or not
- schedule_playoff: This column is a boolean. FALSE is regular season and TRUE is playoffs
- team_home: Name of the home team. This column is a string
- score_home: Points scored by the home team. This column is a number
- score_away: Points scored by the away team. This column is a number
- winner: This column will be a feature derived from score_home and score_away columns to that will use one hot encoding - if team_home scores more points this will be a 1 - if team_home scores fewer points it will be a 0
- team_away: Name of the away team. This column is a string
- team_favorite_id: Acronym of the team that was determined most likely to win by the betting market. It is either two or three letters. For the purposes of this project this column will be changed to be either the team_home or team_away name
- team_home_favorite: this will represent the encoded team_favorite_id - if team_home is favored this column will be marked as a 1 - if it's a zero then we know that team_away is favored
- spread_favorite: The number of points that the favored team needs to win by for a bet placed on the spread of the favorite to win. This column will either be a negative number or zero
- over_under_line: The number of points that both teams combined need to score for a bet placed on the 'line' to win. This column is a positive number
- stadium: Name of the venue that the game is played
- stadium_neutral: This column is a boolean. FALSE is not a neutral venue and TRUE is a neutral venue - for the purposes of this project this column will be one hot encoded with a neutral venue being marked as a 1 and non-neutral marked as a 0
- weather_temperature: The temperature in Fahrenheit at the venue where the game is played. This column is a number
- weather_wind_mph: The speed of wind in miles per hour. This column is a number
- weather_humidity: The measurement of water vapor in the air during the game measured as a percentage. This column is a number
- weather_detail: Other information about the weather conditions - if the venue is indoor or the venue has a retractable roof. This column is a string


## Exploratory Data Analysis
### Questions to answer with EDA:
1. Which columns, if any, should I modify the data type to better train my model?
1. Which columns, if any, should I remove from the training and test data so that the model can be effectively trained?
1. Which columns, if any, should I remove or insert derived data for in the case that there is a lot of missing data?
1. What features could it make sense to introduce to improve the training and performance of my model?

In [2]:
from google.colab import drive
drive.mount('/content/drive/')
# change directory to relevant files
%cd /content/drive/MyDrive/vandy/nfl
# print all file names in directory
for file in os.listdir():
  print(file)

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
/content/drive/MyDrive/vandy/nfl
data_dictionary.csv
spreadspoke_scores.csv


In [3]:
scores = pd.read_csv('spreadspoke_scores.csv')

In [4]:
scores.info()
# scores.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13516 entries, 0 to 13515
Data columns (total 17 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   schedule_date        13516 non-null  object 
 1   schedule_season      13516 non-null  int64  
 2   schedule_week        13516 non-null  object 
 3   schedule_playoff     13516 non-null  bool   
 4   team_home            13516 non-null  object 
 5   score_home           13516 non-null  int64  
 6   score_away           13516 non-null  int64  
 7   team_away            13516 non-null  object 
 8   team_favorite_id     11037 non-null  object 
 9   spread_favorite      11037 non-null  float64
 10  over_under_line      11027 non-null  object 
 11  stadium              13516 non-null  object 
 12  stadium_neutral      13516 non-null  bool   
 13  weather_temperature  12309 non-null  float64
 14  weather_wind_mph     12293 non-null  float64
 15  weather_humidity     8468 non-null  

### Answer to Question 1
- I think that it makes sense to modify both team_home and team_away to have the same IDs as team_favorite_id to make it easier for the model to indentify when team_home is the same or different as team_favorite_id
- Currently Pandas is indentifying schedule_date as an object, it could make sense to see if there's a way for Pandas to indentify this as a date
- Currently Pandas is indentifying schedule_week as an object. This is because some of the data in this column is in string format. It could make sense to modify the data of this column that is in a string to only be in int64 format and use the schedule_playoff column to be the soe determination of whether or not the schedule_week is a playoff game
- Currently Pandas is indentifying the over_under_line column as an object data type despite the fact that it should be a float. I will explore how to ensure that this column's data type is correctly identified
- The stadium_neutral column is currently a boolean type, I think I will convert this to use one hot encoding instead

### Answer to Question 2
- Based on my initial examination, I'm not sure if it makes sense to remove any of my columns from the data set on which I will train my model

In [5]:
scores.isna().sum() # number of missing values for each column

schedule_date              0
schedule_season            0
schedule_week              0
schedule_playoff           0
team_home                  0
score_home                 0
score_away                 0
team_away                  0
team_favorite_id        2479
spread_favorite         2479
over_under_line         2489
stadium                    0
stadium_neutral            0
weather_temperature     1207
weather_wind_mph        1223
weather_humidity        5048
weather_detail         10597
dtype: int64

### Answer to Question 3
- the columns with missing data include team_favorite_id, spread_favorite, over_under_line, weather_temperature, weather_wind_mph, weather_humidity, and weather_detail. these columns have many missing values because this data was not collected in earlier seasons. for example, there is minimal team_favorite_id information collected from the 1978 schedule_season and previous to that likely because of the lack of public betting information before that time
- for the missing data I don't think that it makes sense to remove these columns, however it might make sense to only train and test on the observations from the 1979 schedule_season and beyond

### Answer to Question 4
- I think it makes sense to introduce/derive three different target columns for understanding the performance of the model
- The three target columns I am thinking about introducing are derived from score_home, score_away, and over_under_line
- These targets would be one hot encoded as team_home_win, team_home_cover_spread, and cover_line
- team_home_win would be a 1 if team_home wins or a 0 if they lose
- team_home_cover_spread would be a 1 if they cover the spread_favorite and a 0 if they don't
- cover_line would be a 1 if score_home + score_way is greater than the over_under_line and a 0 if it's less than

## Feature Engineering
1. convert `schedule_date` column to actually be read in as a date/time object instead of generic object
1. convert `schedule_week` to only be numbers - this could be more challenging than I initially thought because there will not be direct mappings for the outliers of `Division`, `Wild Card`, `Conference`, and `Superbowl` because the NFL has expanded the numbers of games played during the regular season over the years and added in the `Wildcard` games
1. use one-hot encoding on the `schedule_playoff` column
1. add a target/feature of `winner` to be derived from `score_home` and `score_away` to make it easier to determine how the model performs
1. convert all `team_home` and `team_away` entries to the acronym identifiers

In [6]:
# convert schedule_date to proper date type
scores['schedule_date'] = pd.to_datetime(scores['schedule_date'])
schedule_date_data_type = scores['schedule_date'].dtype
print('data type of schedule_date: ', schedule_date_data_type)

data type of schedule_date:  datetime64[ns]


In [7]:
# convert schedule_week to only be numbers
scores['schedule_week'].value_counts()

2             831
13            829
1             826
14            826
12            825
11            808
3             782
10            772
4             757
9             754
7             752
8             751
5             745
6             740
15            701
16            687
17            527
Division      217
Wildcard      168
Conference    115
Superbowl      57
18             46
Name: schedule_week, dtype: int64

In [8]:
# convert the binary variable to one-hot encoding on the schedule_playoff column
one_hot_encoded = pd.get_dummies(scores['schedule_playoff'], prefix='schedule_playoff')
# concatenate the one-hot encoded column with the original DataFrame
scores_encoded = pd.concat([scores, one_hot_encoded], axis=1)
display(scores_encoded)

,schedule_date,schedule_season,schedule_week,schedule_playoff,team_home,score_home,score_away,team_away,team_favorite_id,spread_favorite,over_under_line,stadium,stadium_neutral,weather_temperature,weather_wind_mph,weather_humidity,weather_detail,schedule_playoff_False,schedule_playoff_True
0,2066-09-02,1966,1,False,Miami Dolphins,14,23,Oakland Raiders,NaN,NaN,NaN,Orange Bowl,False,83.0,6.0,71.0,NaN,1,0
1,2066-09-03,1966,1,False,Houston Oilers,45,7,Denver Broncos,NaN,NaN,NaN,Rice Stadium,False,81.0,7.0,70.0,NaN,1,0
2,2066-09-04,1966,1,False,San Diego Chargers,27,7,Buffalo Bills,NaN,NaN,NaN,Balboa Stadium,False,70.0,7.0,82.0,NaN,1,0
3,2066-09-09,1966,2,False,Miami Dolphins,14,19,New York Jets,NaN,NaN,NaN,Orange Bowl,False,82.0,11.0,78.0,NaN,1,0
4,2066-09-10,1966,1,False,Green Bay Packers,24,3,Baltimore Colts,NaN,NaN,NaN,Lambeau Field,False,64.0,8.0,62.0,NaN,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13511,2023-01-22,2022,Division,True,Buffalo Bills,10,27,Cincinnati Bengals,BUF,-6.0,48.5,Highmark Stadium,False,32.0,4.0,100.0,snow,0,1
13512,2023-01-22,2022,Division,True,San Francisco 49ers,19,12,Dallas Cowboys,SF,-3.5,46.5,Levi's Stadium,False,55.0,19.0,47.0,NaN,0,1
13513,2023-01-29,2022,Conference,True,Kansas City Chiefs,23,20,Cincinnati Bengals,KC,-1.5,48,GEHA Field at Arrowhead Stadium,False,22.0,13.0,55.0,NaN,0,1
13514,2023-01-29,2022,Conference,True,Philadelphia Eagles,31,7,San Francisco 49ers,PHI,-2.5,45.5,Lincoln Financial Field,False,52.0,14.0,48.0,rain,0,1


In [18]:
# add a target/feature of 'winner'
# Derive a new column based on the comparison of two existing columns
def determine_winner(row):
  if row['score_home'] > row['score_away']:
    val = row['team_home']
  elif row['score_home'] == row['score_away']:
    val = 'tie'
  else:
    val = row['team_away']
  return val

scores_encoded['winning_team'] = scores_encoded.apply(determine_winner, axis=1)
scores_encoded['winning_team'].tail(20)

13496    MIA
13497    CAR
13498    PHI
13499    PIT
13500     SF
13501    SEA
13502    WAS
13503    JAX
13504     SF
13505    BUF
13506    CIN
13507    NYG
13508    DAL
13509     KC
13510    PHI
13511    CIN
13512     SF
13513     KC
13514    PHI
13515     KC
Name: winning_team, dtype: object

In [ ]:
# convert all team_home and team_away entries to the acronym identifiers

# home_names = scores_encoded['team_home'].value_counts()
# away_names = scores_encoded['team_away'].value_counts()
# acronyms = scores_encoded['team_favorite_id'].value_counts()
# print('home teams:\n',home_names)
# print('away teams:\n',away_names)
# print('team acronyms:\n',acronyms)

# map of team names and their corresponding IDs
team_ids = {
    'San Francisco 49ers': 'SF',
    'Dallas Cowboys': 'DAL',
    'Pittsburgh Steelers': 'PIT',
    'Green Bay Packers': 'GB',
    'Philadelphia Eagles': 'PHI',
    'Minnesota Vikings': 'MIN',
    'Denver Broncos': 'DEN',
    'Miami Dolphins': 'MIA',
    'Kansas City Chiefs': 'KC',
    'Buffalo Bills': 'BUF',
    'Chicago Bears': 'CHI',
    'New York Giants': 'NYG',
    'Atlanta Falcons': 'ATL',
    'New Orleans Saints': 'NO',
    'New York Jets': 'NYJ',
    'Detroit Lions': 'DET',
    'Cincinnati Bengals': 'CIN',
    'New England Patriots': 'NE',
    'Washington Redskins': 'WAS',
    'Cleveland Browns': 'CLE',
    # should be SD but all the data uses LAC
    'San Diego Chargers': 'LAC',
    'Seattle Seahawks': 'SEA',
    'Tampa Bay Buccaneers': 'TB',
    # should be OAK but all the data uses LVR
    'Oakland Raiders': 'LVR',
    'Indianapolis Colts': 'IND',
    'Los Angeles Rams': 'LAR',
    'Arizona Cardinals': 'ARI',
    # should be HOU but all the data uses TEN
    'Houston Oilers': 'TEN',
    'Carolina Panthers': 'CAR',
    'Jacksonville Jaguars': 'JAX',
    'Baltimore Ravens': 'BAL',
    'Tennessee Titans': 'TEN',
    # should be STL but all the data uses LAR
    'St. Louis Rams': 'LAR',
    'Houston Texans': 'HOU',
    # should be STL but all the data uses ARI
    'St. Louis Cardinals': 'ARI',
    'Baltimore Colts': 'BAL',
    # should be LAR but all the data uses LVR
    'Los Angeles Raiders': 'LVR',
    'Los Angeles Chargers': 'LAC',
    # should be PHX but all the data uses ARI
    'Phoenix Cardinals': 'ARI',
    # 'Boston Patriots': '', the franchise changed the name of their team to the New England Patriots in 1971
    # the data does not have any team_favorite_id listed before the 12/24/78 in the 1978 season (row 2494)
    'Las Vegas Raiders': 'LVR',
    'Washington Football Team': 'WAS',
    'Tennessee Oilers': 'TEN',
    'Washington Commanders': 'WAS',
}

scores_encoded['team_home'] = scores_encoded['team_home'].replace(team_ids)
scores_encoded['team_away'] = scores_encoded['team_away'].replace(team_ids)

scores_encoded.tail()

In [44]:
scores_encoded.drop('favorite_won', axis=1, inplace=True)
# create new target column of favorite_won in the one-hot encoded style
def determine_favorite_won(row):
  # if not(isinstance(row['team_favorite_id'], str)):
  #   return
  return 1 if row['winning_team'] == row['team_favorite_id'] else 0

scores_encoded.loc[2492:, 'favorite_won'] = scores_encoded.apply(determine_favorite_won, axis=1)
# scores_encoded['favorite_won'] = scores_encoded['favorite_won'].astype(int)
scores_encoded.tail()
# before 12/24/1978 (row 2492) the data does not keep track of the team_favorite_id
# consider removing these rows from test and training data sets
# scores_encoded.loc[2485:2495]

,schedule_date,schedule_season,schedule_week,schedule_playoff,team_home,score_home,score_away,team_away,team_favorite_id,spread_favorite,...,stadium,stadium_neutral,weather_temperature,weather_wind_mph,weather_humidity,weather_detail,schedule_playoff_False,schedule_playoff_True,winning_team,favorite_won
13511,2023-01-22,2022,Division,True,BUF,10,27,CIN,BUF,-6.0,...,Highmark Stadium,False,32.0,4.0,100.0,snow,0,1,CIN,0.0
13512,2023-01-22,2022,Division,True,SF,19,12,DAL,SF,-3.5,...,Levi's Stadium,False,55.0,19.0,47.0,NaN,0,1,SF,1.0
13513,2023-01-29,2022,Conference,True,KC,23,20,CIN,KC,-1.5,...,GEHA Field at Arrowhead Stadium,False,22.0,13.0,55.0,NaN,0,1,KC,1.0
13514,2023-01-29,2022,Conference,True,PHI,31,7,SF,PHI,-2.5,...,Lincoln Financial Field,False,52.0,14.0,48.0,rain,0,1,PHI,1.0
13515,2023-02-12,2022,Superbowl,True,PHI,35,38,KC,PHI,-1.0,...,State Farm Stadium,True,76.0,8.0,8.0,retractable (open roof),0,1,KC,0.0


In [ ]:
# create a pie chart to visualize spread favorite points
spread = scores['spread_favorite'].value_counts()
display(spread)

plt.pie(spread, labels=spread.index, shadow = True)

In [ ]:
# create a bar chart to visualize the same spread favorite points
sns.barplot(y=spread.index, x=spread.values, orient='h')

In [ ]:
# visualize relationships that may exist between team_home and spread_favorite
sns.scatterplot(x=scores['team_favorite_id'], y=scores['spread_favorite'])